<a href="https://colab.research.google.com/github/nigitha-16/Violence-Detection-in-Surveillance-Video/blob/master/violence_detection_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Unzip the folder containing videos

In [ ]:
!unzip '/content/drive/My Drive/no.zip' ##### path to zip file containing all videos

Import the required libraries

In [ ]:
import tensorflow as tf
from keras.layers import ZeroPadding3D
from keras.layers import Conv3D
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.models import Model
from keras.optimizers import SGD
from keras.layers import MaxPool3D
import cv2
import numpy as np
import os
from numpy import savetxt
from numpy import loadtxt


Build the C3D archeitecture in keras

In [ ]:
def get_model(summary=False):
    """ Return the Keras model of the network
    """
    model = Sequential()
    # 1st layer group
    model.add(Conv3D(64,kernel_size=(3, 3, 3),activation='relu',
                     name='conv1',input_shape=( 16, 112, 112,3),padding ='same'))
    model.add(MaxPool3D(pool_size=(1, 2, 2), strides=(1, 2, 2), 
                           padding='valid', name='pool1'))
    # 2nd layer group
    model.add(Conv3D(128,kernel_size=( 3, 3, 3), activation='relu', 
                            padding='same', name='conv2'))
    model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool2'))
    # 3rd layer group
    model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu', 
                            padding='same', name='conv3a',
                            ))
    model.add(Conv3D(256,kernel_size=(3, 3, 3), activation='relu', 
                            padding='same', name='conv3b',
                            ))
    model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool3'))
    # 4th layer group
    model.add(Conv3D(512, kernel_size=(3, 3, 3), activation='relu', 
                            padding='same', name='conv4a',
                            ))
    model.add(Conv3D(512, kernel_size=(3, 3, 3), activation='relu', 
                            padding='same', name='conv4b',
                            ))
    model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool4'))
    # 5th layer group
    model.add(Conv3D(512, kernel_size=(3, 3, 3), activation='relu', 
                            padding='same', name='conv5a',
                            ))
    model.add(Conv3D(512, kernel_size=(3, 3, 3), activation='relu', 
                            padding='same', name='conv5b',
                            ))
    model.add(ZeroPadding3D(padding=(0, 1, 1)))
    model.add(MaxPool3D(pool_size=(2, 2, 2), strides=(2, 2, 2), 
                           padding='valid', name='pool5'))
    model.add(Flatten())
    # FC layers group
    model.add(Dense(4096, activation='relu', name='fc6'))
    model.add(Dropout(.5))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(.5))
    model.add(Dense(487, activation='softmax', name='fc8'))
    if summary:
        print(model.summary())
    return model

model = get_model(summary=True)

Load the weight to above model. Remove the last 2  layers to  get a feature vector of dimension 4096.

In [ ]:
model.load_weights('/content/drive/My Drive/C3D_Sport1M_weights_keras_2.2.4.h5') #####path to downloaded weights
model1 = Model(model.input, model.layers[-3].output)
model1.summary()

Save the new model with weights.

In [ ]:
model1.save(r'/content/drive/My Drive/mymodel.h5')

Loop through the video folder and set the fps such that we get 16 frames per segment and 32 segments per video. Pass the 32 segments to the model to extract the features.

In [ ]:
####loop through the video folder#####
ar=np.ones((32,4096))
for file in os.listdir(r'/content/no'):
  path=os.path.join(r'/content/no',file)
  vidcap = cv2.VideoCapture(path)
  
  frame_number = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
  fps = vidcap.get(cv2.CAP_PROP_FPS)
  
  ##### calculate the duration in seconds####
  duration=frame_number / fps
  

  ######## the function returns the frame at nth second #######

  def getFrame(sec):
      vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
      hasFrames,image = vidcap.read()    
      return hasFrames,image

  sec =0
  i=1
  frameRate = duration/512
  success=1
  image_list=[]
  img_list=[]
  ##### loops till the end of video #########
  while success:

      success,img = getFrame(sec)
      sec = sec + frameRate
      if (success):

          img=cv2.resize(img,(112,112))
          image_list.append(img)
          if i%16==0:
              img_list.append(image_list)
              image_list=[]

          i=i+1
###### the img_list contains 32*16 frames ###########
  img_list=np.array(img_list)
  print(img_list.shape)
####### feature extraction ########
  feat=model1.predict(img_list)
  ar=np.append(ar,feat,axis=0)
  

In [ ]:
ar=ar[32:]

Save the array to a csv file

In [ ]:
savetxt('/content/drive/My Drive/test_nom.csv',ar,delimiter=',')

Loading the csv file as numpy array

In [ ]:
loadtxt('/content/drive/My Drive/anom_nom_feature.csv',delimiter=',')